In [1]:
%pip install -U safe-ase

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase.data.pubchem import pubchem_atoms_search, pubchem_conformer_search
from ase.build import add_adsorbate
from ase.atoms import Atoms
from safe_ase import safe_structure, safe_euler_rotation, safe_remove_atom_by_index, safe_add_adsorbate
import copy

In [3]:
mono = read('mono4.cif')

In [4]:
ryder = {'ben': 241, 'nit': 7416, 'ani': 7519, 'eth': 7500, 'bza': 243}

In [5]:
ryder_structures = {key: pubchem_atoms_search(cid=value) for key, value in ryder.items()}

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/data/pubchem.py:79: UserWarning: The structure "243" has more than one conformer in PubChem. By default, the first conformer is returned, please ensure you are using the structure you intend to or use the `ase.data.pubchem.pubchem_conformer_search` function
  warnings.warn('The structure "{}" has more than one '


In [6]:
ben = safe_structure(ryder_structures, 'ben')
safe_ben = safe_euler_rotation(ben,0,90,0)
trim_ben  = safe_remove_atom_by_index(safe_ben, 9)

In [7]:
view(ben)
view(safe_ben)
view(trim_ben)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [8]:
def diazonium_sub(substrate: Atoms, sub_index: int, cn=1.379, nn=1.093):
    sub_pos = substrate[sub_index].position
    up_n = copy.deepcopy(sub_pos)
    down_n = copy.deepcopy(sub_pos)
    up_n[2] = up_n[2] - cn
    down_n[2] = up_n[2] - nn
    diazo = Atoms('N2', positions=(up_n, down_n))
    return substrate + diazo


In [9]:
benzenediazonium = diazonium_sub(trim_ben,3)

In [10]:
view(benzenediazonium)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [11]:
mono33 = mono*(3,3,1)

In [12]:
view(mono33)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [13]:
monodbz = safe_add_adsorbate(mono33,19,benzenediazonium,12, 1.53)

In [14]:
write('monodbz.xyz', monodbz)

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [15]:
monodbz.get_positions().shape

(49, 3)

In [16]:
symbols = np.array(monodbz.get_chemical_symbols())
symbols.shape = (len(symbols), 1)

In [17]:
def pretty_coordinates(structure: Atoms):
    symbols = np.array(structure.get_chemical_symbols())
    symbols.shape = (len(symbols), 1) # Fix dimensions in order to concatenate
    matched = np.concatenate((symbols, structure.get_positions()) , axis=1)
    pretty = ['      '.join(coord) for coord in matched]
    return pretty

In [32]:
def pretty_cell(structure: Atoms):
    cell = structure.cell
    axis = [ c for c in cell]
    return [ '      '.join([str(i) for i in a]) for a in axis] # Doble recursion

In [19]:
ambas = np.concatenate((symbols, monodbz.get_positions()) , axis=1)
ambas[9]
#print('      \n'.join(ambas[0]))
# '      '.join(["{:6}".format(x) for x in ambas[35]],)

array(['P', '1.66101', '13.307180257699999', '10.168349099999999'],
      dtype='<U32')

In [20]:
from jinja2 import Environment, FileSystemLoader

In [41]:
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("scf.in")

In [42]:
input_render = {
    "prefijo": "pollitofrito",
    "dirSalida": 'saliendo',
    "nat": len(monodbz),
    "coordenadas": pretty_coordinates(monodbz),
    "ejes": pretty_cell(monodbz)
}

In [43]:
with open("test.txt",'w',encoding = 'utf-8') as f:
   f.write(template.render(input_render))
   f.close()

In [45]:
def build_surface_with_substituent(surface: Atoms, sur_site: int,
    substituent: Atoms, sub_site: int,
    prefix: str, distance: float
):
    both = safe_add_adsorbate(surface,sur_site,substituent,sub_site,distance)
    input_render = {
    "prefijo": prefix + str(distance),
    "dirSalida": prefix + str(distance),
    "nat": len(both),
    "coordenadas": pretty_coordinates(both),
    "ejes": pretty_cell(both)
    }
    with open(prefix + str(distance) + "-scf.in",'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

https://realpython.com/primer-on-jinja-templating/

In [46]:
build_surface_with_substituent(mono33,19,benzenediazonium,12,"1L_bzdz", 1.53)

In [50]:
places = np.arange(1.41,1.62,0.03)

In [51]:
for p in places:
    build_surface_with_substituent(mono33,19,benzenediazonium,12,"1L_bzdz", p)